In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST

In [3]:
def topk_mask(x, k):
  shape = x.shape
  x = x.view(-1)
  smallest = torch.topk(x, k)[0][-1]
  x = x * (x >= smallest).float()
  return x.view(shape)

class GEMLinear(nn.Linear):
  def __init__(self, in_features: int, out_features: int, bias: bool = True, topk=1,
  device=None, dtype=None) -> None:
    super().__init__(in_features, out_features, bias, device, dtype)
    self.scores = nn.Parameter(torch.rand_like(self.weight), requires_grad=True)
    self.topk = topk

  def get_mask(self):
    return topk_mask(self.scores, self.topk)
    
  def forward(self, input: torch.Tensor) -> torch.Tensor:
    masked_weight = self.weight * self.get_mask()
    return nn.functional.linear(input, masked_weight * self.mask, self.bias)


In [4]:
train_dataset = MNIST(root='/data/ml_data', train=True, download=True)
val_dataset = MNIST(root='/data/ml_data', train=False, download=True)

OSError: [Errno 30] Read-only file system: '/data'